In [14]:
from arcgis.gis import GIS, Item

# SpatialDataFrame from Item ID

The ArcGIS Python API's [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#spatialdataframe) object is an extension of the [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). As a result, all of the functionality available with a Pandas DataFrame is part of the SpatialDataFrame object with the inclusion of a spatial data type, and methods for interacting with these spatial properties. Since fundamentally still a Pandas DataFrame, the SpatialDataFrame can be used in any data analysis workflows using Pandas DataFrames.

## SpatialDataFrame from Web GIS Item ID

Using a Web GIS Item identifier for a Feature Layer item, a SpatialDataFrame can be created in one line of code. However, to look at all the steps happening, we will incrementally build up to a SpatialDataFrame. Then, we will condense all these steps into one line of code.

Many times, but always, one of the first steps is establishing a connection to an Esri Web GIS using the GIS object. There are a variety of optioons for connecting to a Web GIS depending on whether the Web GIS is ArcGIS Enterprise or ArcGIS Online. These options are well detailed in an [example in the documentation](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/).

Fundamentally, for working with SpatialDataFrames in this notebook, the GIS object stores the propertries for connecting to an Esri WebGIS. This is where we are going to retrieve our data from through a structured REST API for interacting with Esri spatial tables, an ArcGIS Feature Service.

In [15]:
gis = GIS()
gis

GIS @ https://www.arcgis.com version:8.2

Next, simply as a best practice, we are going to save the item identifier in a variable for use later in our notebook.

In [16]:
item_id = '86eefc4a3ccb4b56a5fb2b076588127b'

This is the item identifier collected from the url for the item.

* URL: https://commteamretail.maps.arcgis.com/home/item.html?id=86eefc4a3ccb4b56a5fb2b076588127b
* ID: 86eefc4a3ccb4b56a5fb2b076588127b

### Step by Step

The Web GIS is a custom spatial content management system. All content in our Web GIS is registered as items. Since our starting point is an item identifier, we first need to create an Item object instance.

In [17]:
item = Item(gis, item_id)
item

<Item title:"demoLocations" type:Feature Layer Collection owner:joel5174@esri.com_commteamretail>

This item, when initially published, registers one layer, a REST endpoint for interacting with our Esri spatial table, in our Web GIS. To get access to the data, we need to access the [FeatureLayer](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featurelayer) through the `layers` property.

In [18]:
layer = item.layers[0]
layer

<FeatureLayer url:"https://services.arcgis.com/PMTtzuTB6WiPuNSv/arcgis/rest/services/adf6e4/FeatureServer/0">

This FeatureLayer enables us to interact with the REST endpoint for accessing this spatial table, but we do not yet have the actual data. We need to perform a [query](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.query) to get the actual data. Incidentally, we are explicitly specifiying the output spatial reference, WGS84, using the well-known identifier, 4326. This ensures the geometry coordinates will all be in familiar longitude and latitude.

In [19]:
feature_set = layer.query(out_sr={'wkid': 4326})
feature_set

<FeatureSet> 1808 features

The query returns a [FeatureSet](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featureset) object. This FeatureSet object can be converted to a SpatialDataFrame using the `.df` property.

In [20]:
df = feature_set.sdf
df.head()

,CITY,D_AGGDI_CY,D_AGGNW_CY,D_AMERIND_,D_ASIAN_CY,D_ASSCDEG_,D_AVGDI_CY,D_AVGFMSZ_,D_AVGHHSZ_,D_AVGHINC_,...,LOCNUM,NAICS,SALESVOL,SHAPE,SIC,SQFTCODE,STATE,STREET,ZIP,ZIP4
0,SEASIDE,937237872,-2147483648,425,4388,2668,59435,3.40,2.78,75632,...,666990510,45211101,35495,"{""x"": -121.84299999999993, ""y"": 36.62100000000...",531102,D,CA,CALIFORNIA AVE,93955,3150
1,MARINA,835352352,-2147483648,378,6431,2656,57882,3.37,2.86,72966,...,653371815,45211101,35495,"{""x"": -121.81120000000001, ""y"": 36.66760000000...",531102,D,CA,GENERAL STILLWELL DR,93933,6242
2,CAPITOLA,1493438921,1076047834,501,2246,3969,67479,3.00,2.40,89397,...,423468472,45211101,35495,"{""x"": -121.96509999999991, ""y"": 36.97530000000...",531102,D,CA,41ST AVE,95010,2504
3,WATSONVILLE,597090145,-2147483648,573,1436,1636,49172,4.27,3.91,60171,...,511743478,45211101,35495,"{""x"": -121.77399999999992, ""y"": 36.91540000000...",531102,D,CA,MAIN ST,95076,3755
4,CUPERTINO,-2147483648,-2147483648,259,76813,4541,112177,3.28,2.87,165790,...,404459478,45211101,52059,"{""x"": -122.03619999999995, ""y"": 37.32310000000...",531102,D,CA,STEVENS CREEK BLVD,95014,2123


### One Liner

Above, we performed four steps in separate lines: create the item, retrieve the first layer, query the layer, and convert the layer to a SpatialDataFrame. Thanks to chaining, we can accomplish all of these steps in a single line for brevity if we so desire.

In [21]:
df = Item(gis, item_id).layers[0].query(out_sr={'wkid': 4326}).sdf
df.head()

,CITY,D_AGGDI_CY,D_AGGNW_CY,D_AMERIND_,D_ASIAN_CY,D_ASSCDEG_,D_AVGDI_CY,D_AVGFMSZ_,D_AVGHHSZ_,D_AVGHINC_,...,LOCNUM,NAICS,SALESVOL,SHAPE,SIC,SQFTCODE,STATE,STREET,ZIP,ZIP4
0,SEASIDE,937237872,-2147483648,425,4388,2668,59435,3.40,2.78,75632,...,666990510,45211101,35495,"{""x"": -121.84299999999993, ""y"": 36.62100000000...",531102,D,CA,CALIFORNIA AVE,93955,3150
1,MARINA,835352352,-2147483648,378,6431,2656,57882,3.37,2.86,72966,...,653371815,45211101,35495,"{""x"": -121.81120000000001, ""y"": 36.66760000000...",531102,D,CA,GENERAL STILLWELL DR,93933,6242
2,CAPITOLA,1493438921,1076047834,501,2246,3969,67479,3.00,2.40,89397,...,423468472,45211101,35495,"{""x"": -121.96509999999991, ""y"": 36.97530000000...",531102,D,CA,41ST AVE,95010,2504
3,WATSONVILLE,597090145,-2147483648,573,1436,1636,49172,4.27,3.91,60171,...,511743478,45211101,35495,"{""x"": -121.77399999999992, ""y"": 36.91540000000...",531102,D,CA,MAIN ST,95076,3755
4,CUPERTINO,-2147483648,-2147483648,259,76813,4541,112177,3.28,2.87,165790,...,404459478,45211101,52059,"{""x"": -122.03619999999995, ""y"": 37.32310000000...",531102,D,CA,STEVENS CREEK BLVD,95014,2123


## View Results Inline

Since the SpatialDataFrame is part of the ArcGIS Platform the results can be viewed inline in Jupyter Notebook using a widget displaying the data on a map.

The first step is creating a Web Map object instance using the map method of the GIS object instance we created earlier.

resource: https://developers.arcgis.com/python/guide/visualizing-data-with-the-spatially-enabled-dataframe/

In [9]:
webmap = gis.map('Meades Ranch, KS')
webmap

MapView(layout=Layout(height='400px', width='100%'))

There are a multitude of options available for interacting with this web map. Here we are now going to zoom out to get a better feel for the area we are lookin at...

In [10]:
webmap.zoom = 4

...and next we are also going to change the basemap.

In [11]:
webmap.basemap = 'gray'

Finally, we get to the good part, putting all the data from the SpatialDataFrame on the map using the [`plot` method](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.SpatialDataFrame.plot).

In [13]:
df.spatial.plot(map_widget= webmap,renderer_type='o')

True